# Robust 2D point-set registration
This is a notebook with a toy example to show a solution for the point-set registration problem using robust factors.

In [2]:
import gtsam
import libmix4sam
import numpy as np

Construct the example points. First, we create four *reference* points (fixed frame). Then, we use a `Pose2` class to define a transformation, which we then use to create our *current* point set by transforming the reference points.

In [3]:
initialEstimate = gtsam.Values()
initialEstimate.insert(1, gtsam.Pose2(0,0,0))

# Define reference points
fixed = []
fixed.append(gtsam.Point2(10,1))
fixed.append(gtsam.Point2(20,-1))
fixed.append(gtsam.Point2(30,10))
fixed.append(gtsam.Point2(30,-10))

# Define a ground truth transformation and transform the previous targets into the current frame.
T = gtsam.Pose2(0.25,-0.25, 0.18)
T.print("============ Transformation ==============\n")
current = []
for f in fixed:
    current.append(T.inverse().transformFrom(f))
    #print(f"Original Point: {np.array2string(f)}")
    #print(f"Transformed Point: {np.array2string(current[-1], precision=2)}")

============ Transformation ==============
 (0.25, -0.25, 0.18)


Convert the reference points into a GMM.

In [5]:
refPdf = libmix4sam.Mixture()
refNoise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.3]))
for lmk in fixed:
    refPdf.add(libmix4sam.MixComponent(refNoise, 1.0/len(fixed),lmk))

Use the GMM to instantiate a robust noise model (here we use MaxSumMix) and
add each point/target of the current point set to the factor graph with the mixture distribution of the fixed set as noise model.

In [6]:
refNoiseModel = libmix4sam.noiseModelNew.MaxSumMix.Create(refPdf)
currNoise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.3]))
graph = gtsam.NonlinearFactorGraph()
for lmk in current:
    graph.add(libmix4sam.Psr2DPriorFactor(1,lmk,gtsam.Point2(0,0),refNoiseModel,currNoise))

#graph.print("================= GRAPH ==================\n")

Define the optimizer and optimize.

In [7]:
optimizerParameters = gtsam.LevenbergMarquardtParams()
#SILENT = 0, SUMMARY, TERMINATION, LAMBDA, TRYLAMBDA, TRYCONFIG, DAMPED, TRYDELTA
optimizerParameters.setVerbosityLM("SUMMARY")
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initialEstimate, optimizerParameters)

result = optimizer.optimizeSafely();

Initial error: 239.48, values: 1
iter      cost      cost_change    lambda  success iter_time
   0  1.359478e+01    2.26e+02    1.00e-05     1    2.80e-04
   1  1.055623e+01    3.04e+00    1.00e-06     1    8.06e-05
   2  1.055623e+01    1.90e-08    1.00e-07     1    7.38e-05


In [8]:
result.print("================= RESULT =================\n")
T.print(     "=========== GROUND TRUTH WAS =============\n")

================= RESULT =================

Values with 1 values:
Value 1: (gtsam::Pose2)
(0.25, -0.25, 0.18)

=========== GROUND TRUTH WAS =============
 (0.25, -0.25, 0.18)
